<a href="https://colab.research.google.com/github/Elwing-Chou/ximen_ai_20250630/blob/main/ximen_titanic_0707.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ximen_ai_20250630/raw/refs/heads/main/titanic/train.csv"
urlretrieve(url, "train.csv")

url = "https://github.com/Elwing-Chou/ximen_ai_20250630/raw/refs/heads/main/titanic/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7a77b31a6a50>)

In [5]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")



```
資料預處理(x)

把資料分成兩種
1. 數值類型(連續大小):
Age, SibSp, Parch, Ticket(#), Fare
2. 固定類型:
Pclass, Name, Sex, Cabin, Embarked

1. 補上所有空值
2. 固定類型做One-hot
Extra. 想辦法做出更多欄位

```



In [ ]:
# 注意axis=0/1
train_test = pd.concat([train, test], axis=0)